In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap
# !{sys.executable} -m pip install pip rasterstats 

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
# from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [ ]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [ ]:
## create map
Map = geemap.Map(height="350px")
#Map.centerObject(Districts, zoom=12)
Map

In [ ]:
## Load slope data 

slopeThres = 10 # slope in degrees used as threhold for "high slope". "[Landslide] susceptibility grows quickly between 10° and 30° slopes." (https://link.springer.com/article/10.1007/s11069-017-2757-y#Sec9)
slopeThresStr = str(10)

dataset = ee.Image("NASA/NASADEM_HGT/001")
elevation = dataset.select('elevation');
proj = elevation.select(0).projection();
slope = ee.Terrain.slope(elevation)#.setDefaultProjection(proj));
# Map.addLayer(slope, {min: 0, max: 45}, 'Slope (NASADEM)',false);

# SteepSlopes = slopeReprojected.updateMask(slopeReprojected.gte(slopeThres))
# Map.addLayer(SteepSlopes,{}, 'Slope >='+slopeThresStr+' degrees (ALOS)',False)

SteepSlopes = slope.updateMask(slope.gte(slopeThres))
Map.addLayer(SteepSlopes,{}, 'Slope >='+slopeThresStr+' degrees (NASADEM)',True)

DEMproj = slope.projection().crs()
DEMscale = slope.projection().nominalScale()
DEMtrans = slope.projection().transform()

# print(DEMproj)
# print(DEMscale)

In [ ]:
## Calculate and load vegetation cover raster

NDVIthreshold = 0.4 # decimal
year = 2020

yearStr = str(year)
startdate = ''+yearStr+'-01-01'
enddate = ''+yearStr+'-12-31'

s2 = ee.ImageCollection("COPERNICUS/S2")

def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

green = s2.filterDate(startdate, enddate).map(addNDVI)
green = green.qualityMosaic('NDVI').select('NDVI').float();
#green = green.addBands(ee.Image(year).rename('time_start'))
greenmask = green.updateMask(green.select('NDVI').gte(NDVIthreshold))
# greenmask = (greenmask.reproject(**{'crs':DEMproj,'scale':DEMscale})#'crsTransform':DEMtrans,
#              #.reduceResolution(**{'reducer': ee.Reducer.mean()})
#             )

highslopenotgreenmask = SteepSlopes.updateMask(greenmask.unmask().Not())

Map.addLayer(greenmask,{},"green areas")
Map.addLayer(highslopenotgreenmask,{},"steep slopes without veg")

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Compute indicator

In [ ]:
this_indicator = pd.DataFrame() 

In [ ]:
# define calcuation function to get pixel counts, convert to percents and append to data frame
def CountCalcs(FC,DF):
    # reduce images to get not green steep slope and steep slope pixel counts
    pixelcounts = highslopenotgreenmask.reduceRegions(FC,ee.Reducer.count().setOutputs(['NotGreenSteepPixels']),DEMscale)
    pixelcounts = SteepSlopes.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['SteepPixels']),DEMscale)

    # convert pixel counts to area percentages and saves to FC as property
    def toPct(feat):
        pct = (feat.getNumber('NotGreenSteepPixels')).divide(feat.getNumber('SteepPixels'))
        return feat.set({
            'PctNotGreenSteep_'+yearStr+'': pct
      })

    pixelcounts = pixelcounts.map(toPct).select(['geo_id','PctNotGreenSteep_'+yearStr+''])

    # store in df and apend
    df = geemap.ee_to_pandas(pixelcounts)
    df = df.rename(columns={'PctNotGreenSteep_'+yearStr+'': 'GRE_4_7_percentSteepSlopesWOvegetationcover'+yearStr+''})
    DF = DF.append(df)
    return DF

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicator = CountCalcs(boundary_geo_ee,this_indicator)
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicator = CountCalcs(boundary_geo_ee,this_indicator)

In [ ]:
this_indicator

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_v2test.csv') 
cities_indicators#.head()

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = this_indicator,
                                            indicator_name = 'GRE_4_7_percentSteepSlopesWOvegetationcover'+yearStr+'')

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'data/indicators/cities_indicators_v2test.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')